# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Толстов Евгений Дмитриевич`

Группа: `DS-11`


## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [ ]:
# put your code here

In [63]:
# ! pip install geopip

In [43]:
from random import uniform
from shapely.geometry import Point, LineString, shape, Polygon
import geopip
import folium

In [44]:
# функция генерации случайных гео-точек внутри заданного полигона

import random
from shapely.geometry import Point

def generate_random(number, polygon):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return points

In [76]:
# создадим рекурсивную функцию проверки нахождения точки в Афганистане.

rand_list = []

def find_taliban_point(point_start):
  kabul_point = (69.092047, 34.501541)

  # создаем полигон по 4 точкам: Кабул, случайной и двум точкам со смешанными координатами

  pol = Polygon([kabul_point,
          point_start, 
          (point_start[0], kabul_point[1]),
          (kabul_point[0], point_start[1])])

  # создаем рандомную точку в полигоне
  random_point = generate_random(1, pol)
  rand_list.append(random_point) 
  
  # проверяем страну. Ошибки при попадании в межнудародные воды
  try:
    country_name = geopip.search(lng = random_point[0].x, lat = random_point[0].y)['NAME']
  except TypeError:
    country_name = 'Not founded, probably international water'
  
  if country_name  == 'Afghanistan':
    print('Gotcha!')
  elif country_name != 'Afghanistan':
    print('No, for now it is: ', country_name)
    find_taliban_point((random_point[0].x,random_point[0].y))

  return random_point

In [82]:
# запуск

x, y = uniform(-180,180), uniform(-90, 90)
point = (x,y)

rand_list = []
lst = len(rand_list)

start = find_taliban_point(point)

afgan_point = rand_list[lst - 1][0]

No, for now it is:  Not founded, probably international water
No, for now it is:  Saudi Arabia
No, for now it is:  Saudi Arabia
No, for now it is:  Iran (Islamic Republic of)
No, for now it is:  Iran (Islamic Republic of)
No, for now it is:  Iran (Islamic Republic of)
Gotcha!


In [83]:
# Визуализация. Точки кликабельны

m = folium.Map(
    location=[rand_list[lst - 1][0].y, rand_list[lst - 1][0].x],
    zoom_start=4,
    tiles='Stamen Terrain'
)

tooltip = 'Click me!'

for i in range(len(rand_list)):
  folium.Marker([rand_list[i][0].y, rand_list[i][0].x],popup='<i>Its point N:{}</i>'.format(i), tooltip=tooltip).add_to(m)
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

**1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости**


__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
# put your code here

In [84]:
# pip install OSMPythonTools
# pip install country_list
# pip install rtree

In [210]:
# !curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
# %cd spatialindex-src-1.8.5
# !./configure
# !make
# !make install
# !ldconfig
# !pip install rtree==0.8.3

In [2]:
import pandas as pd

In [3]:
import overpy
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

In [4]:
# загрузим список стран с кодами, среди них найдем нужные areaId 

from country_list import countries_for_language
countries = tuple(countries_for_language('en'))

In [6]:
cntr = []
codes = []
nominatim = Nominatim()
for i in countries:
  area = nominatim.query(i[1]).areaId()
  try:
    if area % 10 == 0:
      codes.append(i[0])
      cntr.append(i[1])
  except TypeError:
    pass

target_cntr = pd.DataFrame()
target_cntr['country_name'] = cntr
target_cntr['country_code'] = codes

In [7]:
print(target_cntr.shape)
target_cntr.head()

(28, 2)


,country_name,country_code
0,Åland Islands,AX
1,Antigua & Barbuda,AG
2,Australia,AU
3,Azerbaijan,AZ
4,Bangladesh,BD


In [111]:
# итого получили 28 стран

In [78]:
# подсчет объектов жилой недвижимости

houses_cnt = []

for i in target_cntr.country_code:
  try:
    r = api.query("""
        area["ISO3166-1"={}][admin_level=2];
        (node["building"~"apartments|house"](area);
        way["building"~"apartments|house"](area);
        rel["building"="apartments"](area);
        );
        out center;
    """.format(i))

    coords  = []
    coords += [(float(node.lon), float(node.lat)) 
              for node in r.nodes]
    coords += [(float(way.center_lon), float(way.center_lat)) 
              for way in r.ways]
    coords += [(float(rel.center_lon), float(rel.center_lat)) 
              for rel in r.relations]

    cnt = len(coords)
    houses_cnt.append(cnt)
  except:
    print('Error on cntry: ', i)


Error on cntry:  BR
Error on cntry:  BQ
Error on cntry:  ST
Error on cntry:  TK
Error on cntry:  UZ


In [80]:
# Пробегая в цикле бывают ошибки на отдельных странах, их обсчитывал отдельно

In [85]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "BR"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

BR_cnt = len(coords)

In [87]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "BQ"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

BQ_cnt = len(coords)

In [88]:

api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "ST"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

ST_cnt = len(coords)

In [89]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "TK"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

TK_cnt = len(coords)

In [90]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "UZ"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

UZ_cnt = len(coords)

In [102]:
target_cntr[target_cntr['country_code'].isin(['BR','BQ','ST','TK','UZ'])]

,country_name,country_code
5,Brazil,BR
8,Caribbean Netherlands,BQ
22,São Tomé & Príncipe,ST
26,Tokelau,TK
27,Uzbekistan,UZ


In [103]:
houses_cnt_tst = houses_cnt.copy()

houses_cnt_tst.insert(5, BR_cnt)
houses_cnt_tst.insert(8, BQ_cnt)
houses_cnt_tst.insert(22, ST_cnt)
houses_cnt_tst.insert(26, TK_cnt)
houses_cnt_tst.insert(27, UZ_cnt)

In [104]:
target_cntr['country_houses_cnt'] = houses_cnt_tst

In [109]:
target_cntr.sort_values(by = ['country_houses_cnt'], ascending = False).head(1)

,country_name,country_code,country_houses_cnt
5,Brazil,BR,8005


In [112]:
# Найдем расстояние от точки в Афганистане до 5 банкоматов в бразилии

In [220]:
# найдем банкоматы

api = overpy.Overpass()
r = api.query("""
area["ISO3166-1"="BR"][admin_level=2];
(node["amenity"="atm"](area);
 way["amenity"="atm"](area);
 rel["amenity"="atm"](area);
);
out center;
""")
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [39]:
from rtree import index
from rtree.index import Rtree

In [223]:
# находим ближайшие к точке в афганистане 5 банкоматов

r_tree = Rtree()

for i, j in enumerate(coords):
    r_tree.insert(i, j+j, j)
    
five_atm = list(r_tree.nearest([afgan_point.x, afgan_point.y], 5, objects = 'raw'))

In [58]:
import math

def haversine(coord1, coord2):
    R = 6_372.8  # Earth radius in km
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [239]:
sum = 0
for i in five_atm:
    sum += haversine(i, (afgan_point.x, afgan_point.y))

In [240]:
sum

58358.3354509871

In [86]:
## Совокупное расстояние составило 58 358.34 км

**2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице**


In [19]:
# подсчет аптек в столице
api = overpy.Overpass()
pharm_cnt = []

for i in target_cntr.country_code:
  try:
    r = api.query("""
        area["ISO3166-1"={}][admin_level=2];
        (node["amenity" = "pharmacy"](area);
        );
        out center; """.format(i))

    coords  = []
    coords += [(float(node.lon), float(node.lat)) 
              for node in r.nodes]
    coords += [(float(way.center_lon), float(way.center_lat)) 
              for way in r.ways]
    coords += [(float(rel.center_lon), float(rel.center_lat)) 
              for rel in r.relations]

    cnt = len(coords)
    pharm_cnt.append(cnt)
  except:
    print('Error on cntry: ', i)

Error on cntry:  AG
Error on cntry:  BR
Error on cntry:  BQ
Error on cntry:  KM
Error on cntry:  TZ
Error on cntry:  UZ


In [21]:
pharm_cnt_save = pharm_cnt.copy()

In [22]:
api = overpy.Overpass()

r = api.query("""
    area["ISO3166-1"= "AG"][admin_level=2];
    (node["amenity" = "pharmacy"](area);
    );
    out center; """)

coords  = []
coords += [(float(node.lon), float(node.lat)) 
          for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
          for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
          for rel in r.relations]

AG_cnt = len(coords)

In [23]:
api = overpy.Overpass()

r = api.query("""
    area["ISO3166-1"= "BR"][admin_level=2];
    (node["amenity" = "pharmacy"](area);
    );
    out center; """)

coords  = []
coords += [(float(node.lon), float(node.lat)) 
          for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
          for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
          for rel in r.relations]

BR_cnt = len(coords)

In [24]:
api = overpy.Overpass()

r = api.query("""
    area["ISO3166-1"= "BQ"][admin_level=2];
    (node["amenity" = "pharmacy"](area);
    );
    out center; """)

coords  = []
coords += [(float(node.lon), float(node.lat)) 
          for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
          for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
          for rel in r.relations]

BQ_cnt = len(coords)

In [25]:
api = overpy.Overpass()

r = api.query("""
    area["ISO3166-1"= "KM"][admin_level=2];
    (node["amenity" = "pharmacy"](area);
    );
    out center; """)

coords  = []
coords += [(float(node.lon), float(node.lat)) 
          for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
          for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
          for rel in r.relations]

KM_cnt = len(coords)

In [26]:
api = overpy.Overpass()

r = api.query("""
    area["ISO3166-1"= "TZ"][admin_level=2];
    (node["amenity" = "pharmacy"](area);
    );
    out center; """)

coords  = []
coords += [(float(node.lon), float(node.lat)) 
          for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
          for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
          for rel in r.relations]

TZ_cnt = len(coords)

In [28]:
api = overpy.Overpass()

r = api.query("""
    area["ISO3166-1"= "UZ"][admin_level=2];
    (node["amenity" = "pharmacy"](area);
    );
    out center; """)

coords  = []
coords += [(float(node.lon), float(node.lat)) 
          for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
          for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
          for rel in r.relations]

UZ_cnt = len(coords)

In [29]:
target_cntr[target_cntr['country_code'].isin(['AG','BR','BQ','KM','TZ','UZ'])]

,country_name,country_code
1,Antigua & Barbuda,AG
5,Brazil,BR
8,Caribbean Netherlands,BQ
10,Comoros,KM
25,Tanzania,TZ
27,Uzbekistan,UZ


In [30]:
pharm_cnt.insert(1, AG_cnt)
pharm_cnt.insert(5, BR_cnt)
pharm_cnt.insert(8, BQ_cnt)
pharm_cnt.insert(10, KM_cnt)
pharm_cnt.insert(25, TZ_cnt)
pharm_cnt.insert(27, UZ_cnt)

In [31]:
target_cntr['pharm_cnt'] = pharm_cnt

In [36]:
target_cntr.sort_values(by = ['pharm_cnt'], ascending=False).head()

,country_name,country_code,pharm_cnt
5,Brazil,BR,6796
21,Portugal,PT,2113
27,Uzbekistan,UZ,1931
2,Australia,AU,1844
25,Tanzania,TZ,1814


In [37]:
# найдем школы

api = overpy.Overpass()
r = api.query("""
area["ISO3166-1"="BR"][admin_level=2];
(node["amenity"="school"](area);
 way["amenity"="school"](area);
 rel["amenity"="school"](area);
);
out center;
""")
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [53]:
r_tree = Rtree()

for i, j in enumerate(coords):
    r_tree.insert(i, j+j, j)
    
five_schools = list(r_tree.nearest([afgan_point.x, afgan_point.y], 5, objects = 'raw'))

In [59]:
sum = 0
for i in five_schools:
    sum += haversine(i, (afgan_point.x, afgan_point.y))

In [61]:
sum

59307.61754090291

In [62]:
## Cовокупное расстояние составило 59 307 км.

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
# put your code here